In [1]:
%matplotlib widget
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(font_scale=1.5)
sns.set_style("whitegrid", {'grid.linestyle':'--'})

In [6]:
title = pd.read_csv('./data/title.basics.tsv.gz', sep='\t', low_memory=False)
rating = pd.read_csv('./data/title.ratings.tsv.gz', sep='\t')
title_w_rating = title.merge(rating, on='tconst')
print(title_w_rating.shape)
title_w_rating.head()

(1015586, 11)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5.6,1575
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",6.1,189
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",6.5,1239
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short",6.2,117
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",6.1,1980


In [46]:
# some data cleaning
movie = title_w_rating[title_w_rating['titleType'] == 'movie']
print(movie.shape)

mask = (movie['runtimeMinutes'] != '\\N')
movie = movie.loc[mask]
movie['runtimeMinutes'] = movie['runtimeMinutes'].astype('int')
mask = movie['runtimeMinutes'] < 600
movie = movie.loc[mask]
mask = (movie['startYear'] != '\\N')
movie = movie.loc[mask]
movie['startYear'] = movie['startYear'].astype('int')
mask = movie['startYear'] > 1980
movie = movie.loc[mask]

(243573, 11)


In [49]:
plt.close('all')
sns.distplot(movie['runtimeMinutes'], kde=False)
plt.tight_layout()
plt.show()

FigureCanvasNbAgg()

In [39]:
plt.close('all')
sns.distplot(movie['averageRating'], kde=False)
plt.tight_layout()
plt.show()

FigureCanvasNbAgg()

Sample size, for the equilvant test, calculated by the R power analyis package `powerTOSTtwo`, with `alpha = 0.05`, `power = 0.9`, and upper/lower bound d of 0.05, is 8658.
~~~R
powerTOSTtwo(alpha=0.05, 
             statistical_power=0.90, 
             low_eqbound_d=-0.05, 
             high_eqbound_d=0.05)
~~~